In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Input , Dropout
from keras.layers import Flatten
from keras.layers import concatenate
from keras.layers.embeddings import Embedding
from keras.models import Model
# from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
import pickle
from keras.layers import LSTM
from keras.preprocessing.text import text_to_word_sequence
import tensorflow as tf
from keras.callbacks import ModelCheckpoint,TensorBoard,ReduceLROnPlateau, EarlyStopping
# from keras.layers.normalization import BatchNormalization
from sklearn.feature_extraction.text import TfidfVectorizer
import seaborn as sns
from keras.regularizers import l2
from sklearn.metrics import roc_auc_score
from keras.models import load_model
from IPython.display import Image
from scipy.sparse import hstack
from keras.layers import Conv1D
from sklearn.feature_extraction.text import CountVectorizer
from prettytable import PrettyTable

from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Conv1D, Embedding, Flatten, Dense, Input, Concatenate, MaxPool1D, Dropout
from tensorflow import keras
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
!pip install tensorflow_text

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/spam or ham/preprocessed_data.csv")
df.shape

(5572, 3)

In [ ]:
df.head(3)

,Unnamed: 0,target,clean_text
0,0,ham,go jurong point crazi avail bugi n great world...
1,1,ham,ok lar joke wif u oni
2,2,spam,free entri 2 wkli comp win fa cup final tkt 21...


In [ ]:
X = df[["clean_text"]]
y = df[["target"]]

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42,stratify = y)

In [ ]:
X_train = list(X_train["clean_text"].values)
X_test = list(X_test["clean_text"].values)

In [ ]:
string = X_train[0]
string

'beauti life next second hide thousand secret wish everi second wonder ur life gud n8 '

In [ ]:
X_train_new = []
for i in X_train:
  X_train_new.append(str(i))

X_test_new = []
for i in X_test:
  X_test_new.append(str(i))

In [ ]:
# %%time.
tokenizer = Tokenizer(filters = '!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n')
tokens = tokenizer.fit_on_texts(X_train_new)

X_train = tokenizer.texts_to_sequences(X_train_new)
X_test = tokenizer.texts_to_sequences(X_test_new)


In [ ]:
# Adding 1 because of  reserved 0 index
vocab_size = len(tokenizer.word_index) + 1                          

# from EDA anything which is above length 250 is considereed as outlier
X_train = pad_sequences(X_train,maxlen = 250,padding="post")
X_test = pad_sequences(X_test,maxlen=250,padding="post")

In [ ]:
type(X_train)

numpy.ndarray

In [ ]:
# string = "hello world"
tokens = tokenizer.texts_to_sequences([string])
tokens

[[413, 112, 132, 414, 1301, 3120, 606, 128, 129, 414, 213, 7, 112, 194, 3121]]

In [ ]:
def create_embedding_matrix(word_index, embedding_dim):
    vocab_size = len(word_index) + 1  
    # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    with open("/content/drive/MyDrive/glove_vectors/glove.6B.100d.txt",'r',encoding='utf8') as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [ ]:
embedding_dim = 100
embedding_matrix = create_embedding_matrix(tokenizer.word_index,embedding_dim)

In [ ]:
# one Hot encoding of the target param
from sklearn.preprocessing import OneHotEncoder as OHE
Encode = OHE(sparse=False)
Y_Train = Encode.fit_transform(y_train)
Y_Test = Encode.fit_transform(y_test)
print(Y_Train.shape, Y_Test.shape)

(4179, 2) (1393, 2)


In [ ]:
Encode.inverse_transform(Y_Train[0].reshape(1, -1))

array([['ham']], dtype=object)

In [ ]:
from sklearn.metrics import f1_score

class Metrics(tf.keras.callbacks.Callback):
    def __init__(self, model):
        self.model=model
    
    def on_train_begin(self, logs={}):
        self.history={'f1_score': []}
        
    def on_epoch_end(self, epoch, logs={}):
        y_pred = np.asarray(self.model.predict(X_test).round())
        logs['f1_score'] = f1_score(Y_Test, y_pred, average='micro', zero_division=1)
        self.history['f1_score'].append(logs.get('f1_score'))
        print('  - f1_score: ', self.history['f1_score'][-1])
            

In [ ]:
in_shape = X_train.shape[1]
input = Input(shape=(in_shape,))
x = Embedding(vocab_size,100, weights = [embedding_matrix], input_length=in_shape,trainable = True)(input)
conv_1d = tf.keras.layers.Conv1D(filters=64, kernel_size=3, padding = 'same',activation='relu')(x)
max_pool = tf.keras.layers.MaxPool1D(2,2)(conv_1d)
x = tf.keras.layers.Conv1D(filters=16, kernel_size=5, padding = 'same',activation='relu')(max_pool)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(256)(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(128)(x)
x = tf.keras.layers.Dense(64)(x)
x = tf.keras.layers.Dense(32)(x)
out = tf.keras.layers.Dense(2, activation='softmax')(x)
model_1 = tf.keras.Model(inputs=input, outputs=out)

In [ ]:
print(model_1.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 250)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 250, 100)          697100    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 250, 64)           19264     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 125, 64)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 125, 16)           5136      
_________________________________________________________________
flatten_1 (Flatten)          (None, 2000)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               5122

In [ ]:
X_train[1].shape

(250,)

In [ ]:
# for saving model , stoping early and chaning learning rate
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
import keras
import datetime
# opt = keras.optimizers.Adam(learning_rate=0.0001)
earlystop = EarlyStopping(monitor='acc', patience=5, verbose=1,mode='max')
reduce_lr = ReduceLROnPlateau(monitor='acc', factor=0.9,patience=2, verbose= 1,mode='max')

log_dir="logs/fit/" + datetime.datetime. now().strftime("%Y%m%d-%H%M%S") 
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)

filepath= "./MODEL"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='acc', verbose=1, 
                             save_best_only=True, mode='auto')

metric_ = Metrics(model_1)
call_back_list=[checkpoint,earlystop,reduce_lr,tensorboard_callback,metric_]
model_1.compile(optimizer="Adam", loss='categorical_crossentropy',metrics=["acc"])


In [ ]:

history = model_1.fit(X_train,Y_Train,epochs=100,batch_size = 8,validation_split=0.2,callbacks=call_back_list)


Epoch 1/100
418/418 [==============================] - 38s 13ms/step - loss: 0.1748 - acc: 0.9399 - val_loss: 0.0664 - val_acc: 0.9737

Epoch 00001: acc improved from -inf to 0.93987, saving model to ./MODEL
INFO:tensorflow:Assets written to: ./MODEL/assets
  - f1_score:  0.968413496051687
Epoch 2/100
418/418 [==============================] - 4s 9ms/step - loss: 0.0505 - acc: 0.9844 - val_loss: 0.0868 - val_acc: 0.9689

Epoch 00002: acc improved from 0.93987 to 0.98445, saving model to ./MODEL
INFO:tensorflow:Assets written to: ./MODEL/assets
  - f1_score:  0.9691313711414213
Epoch 3/100
418/418 [==============================] - 4s 9ms/step - loss: 0.0207 - acc: 0.9931 - val_loss: 0.0729 - val_acc: 0.9880

Epoch 00003: acc improved from 0.98445 to 0.99312, saving model to ./MODEL
INFO:tensorflow:Assets written to: ./MODEL/assets
  - f1_score:  0.9784637473079684
Epoch 4/100
418/418 [==============================] - 4s 10ms/step - loss: 0.0020 - acc: 0.9991 - val_loss: 0.1260 - val_a

In [ ]:
y_pred_train = (model_1.predict(X_train))

In [ ]:
y_pred_train=np.argmax(y_pred_train, axis=1)

In [ ]:
le=LabelEncoder()
le.fit(y_train.values)
y_tr = le.transform(y_train.values)
y_te = le.transform(y_test.values)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
from sklearn.metrics import f1_score
print("train_f1_score is ",f1_score(y_tr, y_pred_train, average="macro"))

train_f1_score is  0.9948526825173005


In [ ]:
y_pred_test = (model_1.predict(X_test))
y_pred_test=np.argmax(y_pred_test, axis=1)
print("train_f1_score is ",f1_score(y_te, y_pred_test, average="macro"))

train_f1_score is  0.9602193220435788


In [ ]:
output = []
def reemovNestings(l):
    for i in l:
        if type(i) == list:
            reemovNestings(i)
        else:
            output.append(i)
    return output

In [ ]:
def inference(sentence):

  tokens = tokenizer.texts_to_sequences([sentence])
  tokens_padded = pad_sequences(tokens,maxlen = 250,padding="post")
  # here we get nested list convert to single llist
  # tokens_new = reemovNestings(tokens)
  
  y_pred = (model_1.predict(tokens_padded))
  
  y_pred=np.argmax(y_pred, axis=1)
  if y_pred[0] == 0:
    return "ham"
  else:
    return "spam"
  # return y_pred




In [ ]:
inference("hello world")

'ham'

fine tunning bert for spam classification

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['clean_text'],df['target'], stratify=df['target'])

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 40.3 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
     |████████████████████████████████| 3.3 MB 28.3 MB/s 
     |████████████████████████████████| 636 kB 48.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

TypeError: ignored

In [ ]:
X_train = list(X_train["clean_text"].values)
X_test = list(X_test["clean_text"].values)

KeyError: ignored

In [ ]:
get_sentence_embeding([
    "500$ discount. hurry up", 
    ]
)

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = get_sentence_embeding(text_input)
# print(preprocessed_text)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

ValueError: ignored

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_mask': (None 0           text[0][0]                       
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'default': (None, 7 109482241   keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
len(X_train)

4179

In [ ]:

bert_model = model.fit(X_train, y_train, epochs=10,validation_split=0.1)

ValueError: ignored

In [ ]:
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)

ValueError: ignored